In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from tensorflow.keras.preprocessing import image_dataset_from_directory

import os



In [24]:
from google.colab import drive
drive.mount('/content/drive')


ROOT_DIR = "/content/drive/MyDrive/Colab_Notebooks/ML_INAPP_Proj/ML-in-Apps-2024-AM03-Extracting-Features-from-Medical-Images/dataset/labeled_dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
ROOT_DIR = "./dataset/labeled_dataset/"
tumor_dir = os.path.join(ROOT_DIR,"in_roi_patches")
not_tumor_dir = os.path.join(ROOT_DIR,"not_roi_patches")

In [27]:
BATCH_SIZE = 16
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 3

In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models
train_ds =  image_dataset_from_directory(
    ROOT_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE)

val_ds = image_dataset_from_directory(
    ROOT_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size = BATCH_SIZE)

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

Found 14269 files belonging to 2 classes.
Using 11416 files for training.
Found 14269 files belonging to 2 classes.
Using 2853 files for validation.


In [26]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

# Configurare la policy di mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)


ImportError: cannot import name 'experimental' from 'tensorflow.keras.mixed_precision' (/usr/local/lib/python3.10/dist-packages/keras/api/_v2/keras/mixed_precision/__init__.py)

In [ ]:
from tensorflow.keras.applications import ResNet50
FREEEZE_LAYERS = 10
EPOCHS = 5
LEARNING_RATE = 0.001

base_model = ResNet50(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
                         include_top=False,
                         weights='imagenet')
num_layers = len(base_model.layers)
print(f"Il modello DenseNet121 ha {num_layers} layers.")


for layer in base_model.layers[:FREEEZE_LAYERS]:
    layer.trainable = False

base_model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')  # Configurato per classificazione binaria
    ])
base_model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

base_model.compile(optimizer=optimizer,
                     loss=loss,
                     metrics=['accuracy'])

history = base_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
print("Training completato.")

print("Accuracy sul training set: ", history.history['accuracy'][-1])
print("Loss sul training set: ", history.history['loss'][-1])

print("Accuracy sul validation set: ", history.history['val_accuracy'][-1])
print("Loss sul validation set: ", history.history['val_loss'][-1])


test_loss, test_accuracy = base_model.evaluate(val_ds)


Il modello DenseNet121 ha 175 layers.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 16, 16, 2048)      23587712  
                                                                 
 global_average_pooling2d_2  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23589761 (89.99 MB)
Trainable params: 23522753 (89.73 MB)
Non-trainable params: 67008 (261.75 KB)
_________________________________________________________________
Epoch 1/5
714/714 [==============================] - 859s 1s/step - loss: 0.4916 - accuracy: 0.7770 - val_loss: 67.5901 - val_accuracy: 0.5188
Epoch 2/5
714/714 [======